TODO:
- Finish submission_rows. Combine predictions from site and app models. Provide id for each row.
- why are the predicted values in out.txt negative?
- cross-validation: start with one validation set.

In [1]:
import xlearn as xl

In [2]:
def run_ffm(train_set, validate_set, test_set, model_out, predict_out):
    ffm_model = xl.create_ffm()
    ffm_model.setTrain(train_set)
    ffm_model.setValidate(validate_set)
    
    param = {'task':'binary', 'lr':0.2, 'lambda':0.002}
    ffm_model.fit(param, model_out)
    
    ffm_model.setSigmoid()
    ffm_model.setTest(test_set)
    ffm_model.predict(model_out, predict_out)
    print(f'Prediction written to {predict_out}')

In [3]:
# TODO: modify this function to take predictions from site and app models,
#    and generate rows to write.
def submission_rows(predict_out, id_file, fout):
    with open(predict_out, 'r') as prediction, open(id_file, 'r') as ids, open(fout, 'w') as out:
        out.write('id,click\n')
        for i, val in zip(ids, prediction):
            yield f'{i.strip()},{val.strip()}\n'

In [4]:
#TODO: move this to tools/kaggle_tools
def validate_submission(submission_csv, ids):
    """
    Ensure that the submission file is in the right format, 
    and that it contains required id's.
    """
    pass

In [5]:
train_site = './data/train_site_tiny.ffm'
validate_site = './data/validate_site_tiny.ffm'
test_site = './data/test_site_tiny.ffm'
model_out = './xlearn-out/ffm-site.out'
predict_out = './xlearn-out/out-site.txt'

In [6]:
run_ffm(train_site, validate_site, test_site, model_out, predict_out)

Prediction written to ./xlearn-out/out-site.txt


In [7]:
write_submission(predict_out, test_site + '.id', 'submission-site.csv')

In [8]:
train_app = './data/train_app_tiny.ffm'
validate_app = './data/validate_app_tiny.ffm'
test_app = './data/test_app_tiny.ffm'
model_out = './xlearn-out/ffm-app.out'
predict_out = './xlearn-out/out-app.txt'

run_ffm(train_app, validate_app, test_app, model_out, predict_out)

Prediction written to ./xlearn-out/out-app.txt


In [9]:
write_submission(predict_out, test_app + '.id', 'submission-app.csv')